In [1]:
!pip install sentence-transformers

In [2]:
import json
import csv
from sentence_transformers import SentenceTransformer

In [4]:
with open("/content/extracted_patterns.json", "r", encoding="utf-8") as f:
    patterns = json.load(f)

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
rows = []
patterns_with_embeddings = []

for p in patterns:
    related_patterns = p.get('Related Patterns', [])
    if len(related_patterns) >0 and type(related_patterns[0]) == type({}):
        related_patterns = [i for _,i in related_patterns]

    print(related_patterns)

    text = f"{p.get('Pattern Name', '')} {p.get('Problem', '')} {p.get('Context', '')} {p.get('Solution', '')} {p.get('Result', '')} {p.get('Uses', '')} {' '.join(p.get('Category', []))} {' '.join(related_patterns)}"

    embedding = model.encode(text).tolist()

    rows.append([p.get("Pattern Name", "Unknown")] + embedding)

    p_with_embed = p.copy()
    p_with_embed["embedding"] = embedding
    patterns_with_embeddings.append(p_with_embed)

['LLMs as Content Interpreter', 'LLMs as Explainer']
['LLMs as Knowledge Base', 'Prompt Design Patterns']
['Prompt Design Patterns', 'AI–Human Interaction Patterns', 'Generative AI Patterns']
['Prompt Design Patterns', 'LLM-specific Patterns', 'Knowledge & Reasoning Patterns']
['Planning Patterns', 'LLM-specific Patterns', 'Knowledge & Reasoning Patterns']
['Tools Integration Patterns', 'Prompt Design Patterns', 'LLM-specific Patterns', 'AI–Human Interaction Patterns']
['Agentic AI Patterns', 'Prompt Design Patterns', 'LLM-specific Patterns']
['LLM-based Tool Learning', 'LLMs as Conversational Agent', 'Personalization Pattern']
['Generative AI Patterns', 'Personalization Pattern', 'AI–Human Interaction Patterns']
['Agentic AI Patterns', 'LLM-specific Patterns', 'Tools Integration Patterns', 'AI–Human Interaction Patterns']
['Agentic AI Patterns', 'Prompt Design Patterns', 'LLM-specific Patterns']
['Agentic AI Patterns', 'LLM-specific Patterns', 'Context Window Optimization']
['LLM-spec

In [16]:
with open("patterns_embeddings.tsv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter="\t")
    header = ["Pattern Name"] + [f"dim_{i}" for i in range(len(rows[0]) - 1)]
    writer.writerow(header)
    writer.writerows(rows)

In [17]:
with open("patterns_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(patterns_with_embeddings, f, indent=2, ensure_ascii=False)